# Create Study Area

**Timm Nawrocki**  
Alaska Center for Conservation Science  
2019-03-20

In [1]:
# -*- coding: utf-8 -*-
# ---------------------------------------------------------------------------
# Create Study Area
# Author: Timm Nawrocki
# Created on: 2019-03-20
# Usage: Must be executed as a Jupyter Notebook in an ArcGIS Pro Python 3 installation.
# Description: "Create Study Area" generates a polygon of Game Management Unit (GMU 17) plus the Bristol Bay Lowlands and Ahklun Mountains. This script also outputs several supporting polygons that parse the study area into smaller sections.
# ---------------------------------------------------------------------------

## 1. Initialize Environment

In [2]:
# Import packages
import arcpy
import os

# Set overwrite option
arcpy.env.overwriteOutput = True

# Set root directory
drive = 'F:/'
root_directory = os.path.join(drive, 'ACCS_Work/Projects/VegetationEcology/BristolBay_Vegetation/Project_GIS')

# Set arcpy working environment
arcpy.env.workspace = os.path.join(root_directory, 'BristolBay_Vegetation.gdb')

# Define script paths
script_directory = os.path.join(drive, 'Repositories/southwest-alaska-moose/modules')
arcpy_geoprocessing_script = os.path.join(script_directory, 'arcpy_geoprocessing.py')

# Define input datasets
game_management_units = os.path.join(root_directory, 'Data_Input/source_data/boundaries/Alaska_GameManagementUnits.shp')
unified_ecoregions = os.path.join(root_directory, 'Data_Input/source_data/boundaries/Alaska_UnifiedEcoregions.shp')
watersheds = os.path.join(root_directory, 'Data_Input/source_data/inland_waters/NHD_H_02_GDB.gdb/WBD/WBDHU10')

# Define output feature class
gmu_17 = 'SouthwestAlaska_GMU17'
output_ecoregions = 'SouthwestAlaska_BristolBayEcoregions'
study_area = 'SouthwestAlaska_StudyArea'
bristol_lowlands_addition = 'SouthwestAlaska_BristolLowlandsAddition'
ahklun_mountains_addition = 'SouthwestAlaska_AhklunMountainsAddition'

In [3]:
# Import and execute arcpy_geoprocessing.py
try:
    exec(open(arcpy_geoprocessing_script).read())
except:
    print("Error loading arcpy_geoprocessing.py; ensure that script directory is correct:")
    print(script_directory)
    quit()

## 2. Create Study Area Polygons

In [4]:
# Define function to create polygon feature class
def create_study_area(**kwargs):
    """
    Description: creates a polygon feature class with vertices read from an array
    Inputs: 'input_array' -- an array containing game management units, unified ecoregions, and watersheds (10 digit hydrologic units)
            'output_array' -- the output should be an array with a full feature class and a clipped feature class
    """
    # Parse key word argument inputs
    game_management_units = kwargs['input_array'][0]
    unified_ecoregions = kwargs['input_array'][1]
    watersheds = kwargs['input_array'][2]
    gmu_17 = kwargs['output_array'][0]
    output_ecoregions = kwargs['output_array'][1]
    study_area = kwargs['output_array'][2]
    bristol_lowlands_addition = kwargs['output_array'][3]
    ahklun_mountains_addition = kwargs['output_array'][4]
    # Define intermediate datasets
    kaskanak_creek = 'kaskanakCreek'
    ahklun_mountains = 'ahklun_mountains'
    bristol_lowlands = 'bristol_lowlands'
    study_area_initial = 'studyArea_initial'
    study_area_line = 'studyArea_line'
    merge_ahklun = 'merge_ahklun'
    merge_bristol = 'merge_bristol'
    # Select the Ahklun Mountains from the unified ecoregions
    select_ahklun = arcpy.SelectLayerByAttribute_management(unified_ecoregions, 'NEW_SELECTION', "COMMONER = 'Ahklun Mountains'", 'NON_INVERT')
    arcpy.CopyFeatures_management(select_ahklun, ahklun_mountains)
    # Select the Bristol Bay Lowlands from the unified ecoregions
    select_bristol = arcpy.SelectLayerByAttribute_management(unified_ecoregions, 'NEW_SELECTION', "COMMONER = 'Bristol Bay Lowlands'", 'NON_INVERT')
    arcpy.CopyFeatures_management(select_bristol, bristol_lowlands)
    # Merge the selected ecoregions into one feature class
    arcpy.Merge_management([ahklun_mountains, bristol_lowlands], output_ecoregions)
    print('Successfully created output ecoregions feature class...')
    # Select GMU 17 from GMUs
    select_gmu = arcpy.SelectLayerByAttribute_management(game_management_units, 'NEW_SELECTION', "UnitSub IN ('17A', '17B', '17C')", 'NON_INVERT')
    arcpy.CopyFeatures_management(select_gmu, gmu_17)
    print('Successfully created feature class for GMU 17...')
    # Merge the selected ecoregions with GMU 17 and Kaskanak Creek watershed to form study area
    select_watershed = arcpy.SelectLayerByAttribute_management(watersheds, 'NEW_SELECTION', "HUC10 = '1903020612'", 'NON_INVERT')
    arcpy.CopyFeatures_management(select_watershed, kaskanak_creek)
    arcpy.Merge_management([output_ecoregions, gmu_17, kaskanak_creek], study_area_initial)
    # Fill holes in the study area
    arcpy.PolygonToLine_management(study_area_initial, study_area_line, 'IGNORE_NEIGHBORS')
    arcpy.FeatureToPolygon_management(study_area_line, study_area_initial)
    arcpy.Dissolve_management(study_area_initial, study_area)
    print('Successfully created study area feature class...')
    # Isolate areas additional to GMU 17
    arcpy.Merge_management([gmu_17, kaskanak_creek, bristol_lowlands], merge_bristol)
    arcpy.Erase_analysis(study_area, merge_bristol, ahklun_mountains_addition)
    print('Successfully created Ahklun Mountains addition area...')
    arcpy.Merge_management([gmu_17, ahklun_mountains], merge_ahklun)
    arcpy.Erase_analysis(study_area, merge_ahklun, bristol_lowlands_addition)
    print('Successfully created Bristol Bay Lowlands addition area...')
    print('----------')
    # Delete intermediate dataset
    arcpy.Delete_management(kaskanak_creek)
    arcpy.Delete_management(ahklun_mountains)
    arcpy.Delete_management(bristol_lowlands)
    arcpy.Delete_management(study_area_initial)
    arcpy.Delete_management(merge_ahklun)
    out_process = arcpy.Delete_management(merge_bristol)
    return out_process

In [5]:
# Define input and output arrays
study_area_inputs = [game_management_units, unified_ecoregions, watersheds]
study_area_outputs = [gmu_17, output_ecoregions, study_area, bristol_lowlands_addition, ahklun_mountains_addition]

# Create key word arguments
study_area_kwargs = {'input_array' : study_area_inputs,
                     'output_array' : study_area_outputs
                    }

# Process the create polygon function with the point array
arcpy_geoprocessing(create_study_area, **study_area_kwargs)

SouthwestAlaska_GMU17 already exists and will be overwritten.
SouthwestAlaska_BristolBayEcoregions already exists and will be overwritten.
SouthwestAlaska_StudyArea already exists and will be overwritten.
SouthwestAlaska_BristolLowlandsAddition already exists and will be overwritten.
SouthwestAlaska_AhklunMountainsAddition already exists and will be overwritten.
Successfully created output ecoregions feature class...
Successfully created feature class for GMU 17...
Successfully created study area feature class...
Successfully created Ahklun Mountains addition area...
Successfully created Bristol Bay Lowlands addition area...
----------
Succeeded at Sunday, March 24, 2019 1:59:45 PM (Elapsed Time: 0.56 seconds)
